<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3fd76b6d0398855be9c30e639465de919992e9d6ea8f8da8fe131ff25fcc325e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 15:01:03
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 2.19 K (0.02%)
Current PnL: -16.74 L (-11.26%)
CY Booked + Current PnL: -5.30 L (-3.57%)
-------------------
Total profit:  2.09 L
Total loss:  -18.83 L
-------------------
Total Booked + Current PnL: 21.68 L (17.81%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.26 L (56.88%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.50,196420.0,31750.0,11589.0,0.16,19.28,5.90,26.31,89.0,2.74,1.44,28.80,X40N,NTT,AC
76,TTKPRESTIG,770.00,103.42,54.0,M-SC,3.70,87600.0,-13177.0,13271.0,-0.73,-13.08,15.15,0.09,245.0,-0.99,0.64,15.09,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.44,59.0,H-MC,2.68,212790.0,21048.0,16704.0,-0.53,10.98,7.85,19.69,99.0,1.26,1.56,18.52,XY25,NTT,AC
49,NATIONALUM,244.55,-44.49,63.0,H-MC,7.92,111470.0,10966.0,18147.0,-0.46,10.91,16.28,28.96,79.0,0.60,0.82,48.58,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,57.0,H-LC,4.96,219991.0,9625.0,19161.0,-0.34,4.58,8.71,13.68,37.0,0.50,1.61,21.85,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-14.99,78.0,H-SC,12.43,227557.0,-43204.0,373148.0,6.30,-15.96,163.98,121.86,133.0,-0.12,1.66,78.01,AR,ATH,IT
78,UNIONBANK,163.00,-11.63,67.0,M-LC,7.06,159623.0,18783.0,25715.0,2.78,13.34,16.11,31.59,66.0,0.73,1.17,43.36,XY24,NTT,BANKS
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85398.0,-16812.0,110206.0,2.26,-16.45,129.05,91.37,208.0,-0.15,0.62,38.63,XY25,NTT,FINANCE
26,GREENPANEL,537.00,235.92,62.0,M-SC,1.77,149865.0,-33213.0,112189.0,2.15,-18.14,74.86,43.14,230.0,-0.30,1.10,40.97,XY24,NTT,MISC
59,SAIL,228.00,48.36,70.0,M-MC,7.56,234192.0,9230.0,158431.0,2.12,4.10,67.65,74.53,192.0,0.06,1.71,37.68,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,QUESS,986.00,-41.60,43.0,M-SC,39.88,58013.0,-6993.0,157923.0,-2.82,-10.76,272.22,232.18,198.0,-0.04,0.42,1.03,XY24,NTT,MISC
7,ATULAUTO,844.00,3797.30,59.0,M-SC,5.38,121564.0,-25804.0,77619.0,-2.47,-17.51,63.85,35.16,236.0,-0.33,0.89,24.17,XY24,NTT,AUTO
80,VAIBHAVGBL,521.00,61.43,64.0,H-SC,6.64,140148.0,-42627.0,153700.0,-2.44,-23.32,109.67,60.77,125.0,-0.28,1.03,25.53,XR,NTT,JEWELLERY
55,RELAXO,1176.00,-33.29,51.0,H-SC,1.90,90926.0,-54234.0,130161.0,-2.37,-37.36,143.15,52.31,136.0,-0.42,0.67,21.71,X40N,NTT,FOOTWEAR
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130910.0,7574.0,126184.0,-2.10,6.14,96.39,108.45,119.0,0.06,0.96,29.87,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.00,-20.65,46.0,H-MC,2.18,136493.0,705.0,42053.0,-0.17,0.52,30.81,31.48,107.0,0.02,1.00,12.70,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.48,59.0,H-MC,1.92,248929.0,1198.0,73857.0,0.09,0.48,29.67,30.30,92.0,0.02,1.82,13.84,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.28,56.0,H-MC,4.08,201825.0,1005.0,67773.0,0.76,0.50,33.58,34.25,80.0,0.01,1.48,5.72,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,103.42,54.0,M-SC,3.70,87600.0,-13177.0,13271.0,-0.73,-13.08,15.15,0.09,245.0,-0.99,0.64,15.09,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,54.0,H-SC,9.41,123201.0,-19278.0,38796.0,0.26,-13.53,31.49,13.70,163.0,-0.50,0.90,45.41,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.24,62.0,H-SC,1.27,226449.0,-43218.0,79552.0,-0.65,-16.03,35.13,13.47,138.0,-0.54,1.66,14.53,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,45.0,H-SC,2.11,145394.0,-30509.0,72537.0,-0.41,-17.34,49.89,23.89,149.0,-0.42,1.06,25.57,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.51,54.0,H-MC,7.43,108851.0,-22984.0,65213.0,-0.08,-17.43,59.91,32.03,98.0,-0.35,0.80,21.92,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,70.0,M-MC,7.56,234192.0,9230.0,158431.0,2.12,4.10,67.65,74.53,192.0,0.06,1.71,37.68,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.50,196420.0,31750.0,11589.0,0.16,19.28,5.90,26.31,89.0,2.74,1.44,28.80,X40N,NTT,AC
83,VOLTAS,1530.00,1.44,59.0,H-MC,2.68,212790.0,21048.0,16704.0,-0.53,10.98,7.85,19.69,99.0,1.26,1.56,18.52,XY25,NTT,AC
17,CAMS,5211.76,0.82,67.0,H-SC,3.46,114517.0,12513.0,31412.0,0.36,12.27,27.43,43.06,122.0,0.40,0.84,31.48,X40N,ATH,MISC
29,HAVELLS,2069.16,0.48,59.0,H-MC,1.92,248929.0,1198.0,73857.0,0.09,0.48,29.67,30.30,92.0,0.02,1.82,13.84,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-44.49,63.0,H-MC,7.92,111470.0,10966.0,18147.0,-0.46,10.91,16.28,28.96,79.0,0.60,0.82,48.58,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,0.65,100372.0,8874.0,69879.0,0.22,9.70,69.62,86.07,223.0,0.13,0.73,45.76,XR,NTT,DURABLES
85,WIPRO,420.00,-9.26,60.0,M-LC,5.63,159297.0,8352.0,101106.0,1.09,5.53,63.47,72.51,53.0,0.08,1.17,11.89,XR,NTT,IT
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130910.0,7574.0,126184.0,-2.10,6.14,96.39,108.45,119.0,0.06,0.96,29.87,AR,ATH,MISC
11,BANKINDIA,190.00,-27.95,66.0,H-MC,6.33,184975.0,5167.0,107249.0,1.06,2.87,57.98,62.52,88.0,0.05,1.35,37.94,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-15.68,44.0,H-LC,6.54,303560.0,-12282.0,124945.0,0.48,-3.89,41.16,35.67,5.0,-0.10,2.22,9.43,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.32,46.0,H-LC,7.76,218944.0,-32824.0,86045.0,0.38,-13.04,39.30,21.13,27.0,-0.38,1.60,17.77,X40,ATH,PAINTS
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.74,230821.0,-11103.0,74371.0,-1.58,-4.59,32.22,26.15,8.0,-0.15,1.69,9.08,X40,ATH,IT
43,JIOFIN,387.00,0.06,51.0,H-LC,13.13,271889.0,4524.0,60930.0,-0.38,1.69,22.41,24.48,48.0,0.07,1.99,57.59,XY24,BTT,FINANCE
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200955.0,817.0,20075.0,-0.15,0.41,9.99,10.44,4.0,0.04,1.47,5.99,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.00,69.0,H-LC,1.48,248062.0,1911.0,77296.0,-0.22,0.78,31.16,32.18,16.0,0.02,1.81,37.10,X200,ATH,IT
41,ITC,452.0,-37.92,51.0,H-LC,2.37,200955.0,817.0,20075.0,-0.15,0.41,9.99,10.44,4.0,0.04,1.47,5.99,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.11,52.0,H-LC,4.18,279583.0,14157.0,42636.0,-1.18,5.33,15.25,21.40,11.0,0.33,2.05,13.06,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,57.0,H-LC,4.96,219991.0,9625.0,19161.0,-0.34,4.58,8.71,13.68,37.0,0.50,1.61,21.85,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,54.0,H-LC,5.74,215296.0,10796.0,30615.0,-0.42,5.28,14.22,20.25,10.0,0.35,1.58,14.47,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.80,52.0,H-LC,7.76,228468.0,-5676.0,54307.0,0.07,-2.42,23.77,20.77,86.0,-0.10,1.67,4.33,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297115.0,-48841.0,123689.0,-0.50,-14.12,41.63,21.64,1.0,-0.39,2.17,5.25,X40,ATH,IT
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200955.0,817.0,20075.0,-0.15,0.41,9.99,10.44,4.0,0.04,1.47,5.99,X40,NTT,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.74,230821.0,-11103.0,74371.0,-1.58,-4.59,32.22,26.15,8.0,-0.15,1.69,9.08,X40,ATH,IT
82,VBL,671.64,-15.68,44.0,H-LC,6.54,303560.0,-12282.0,124945.0,0.48,-3.89,41.16,35.67,5.0,-0.10,2.22,9.43,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.60,65.0,H-LC,26.99,166440.0,-19655.0,232051.0,0.33,-10.56,139.42,114.13,15.0,-0.08,1.22,22.57,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200955.0,817.0,20075.0,-0.15,0.41,9.99,10.44,4.0,0.04,1.47,5.99,X40,NTT,FMCG
19,CIPLA,1795.00,-19.44,54.0,H-LC,5.74,215296.0,10796.0,30615.0,-0.42,5.28,14.22,20.25,10.0,0.35,1.58,14.47,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.32,46.0,H-LC,7.76,218944.0,-32824.0,86045.0,0.38,-13.04,39.30,21.13,27.0,-0.38,1.60,17.77,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,57.0,H-LC,4.96,219991.0,9625.0,19161.0,-0.34,4.58,8.71,13.68,37.0,0.50,1.61,21.85,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297115.0,-48841.0,123689.0,-0.50,-14.12,41.63,21.64,1.0,-0.39,2.17,5.25,X40,ATH,IT
39,INFY,2275.00,-16.37,56.0,H-LC,8.17,325954.0,13066.0,158609.0,-0.67,4.18,48.66,54.87,3.0,0.08,2.38,11.10,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200955.0,817.0,20075.0,-0.15,0.41,9.99,10.44,4.0,0.04,1.47,5.99,X40,NTT,FMCG
82,VBL,671.64,-15.68,44.0,H-LC,6.54,303560.0,-12282.0,124945.0,0.48,-3.89,41.16,35.67,5.0,-0.10,2.22,9.43,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.85,260990.0,-629.0,119847.0,0.02,-0.24,45.92,45.57,7.0,-0.01,1.91,11.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,RAJESHEXPO,518.00,1891.16,67.0,L-SC,3.06,54314.0,-82863.0,82954.0,0.06,-60.41,152.73,0.07,267.0,-1.00,0.40,35.08,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,6927.78,61.0,L-SC,5.18,77128.0,-16682.0,93302.0,-0.56,-17.78,120.97,81.67,269.0,-0.18,0.56,49.36,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.29,85267.0,-28282.0,68333.0,1.02,-24.91,80.14,35.27,268.0,-0.41,0.62,108.69,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85398.0,-16812.0,110206.0,2.26,-16.45,129.05,91.37,208.0,-0.15,0.62,38.63,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-5.18,45.0,H-SC,12.47,88253.0,-12710.0,107236.0,-1.10,-12.59,121.51,93.63,148.0,-0.12,0.65,31.45,SR,ATH,CHEMICALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-14.99,78.0,H-SC,12.43,227557.0,-43204.0,373148.0,6.30,-15.96,163.98,121.86,133.0,-0.12,1.66,78.01,AR,ATH,IT
63,SHALBY,327.00,1107.34,69.0,M-SC,4.59,164179.0,-16964.0,62437.0,0.68,-9.37,38.03,25.10,235.0,-0.27,1.20,31.57,XY24,NTT,HEALTHCARE
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85398.0,-16812.0,110206.0,2.26,-16.45,129.05,91.37,208.0,-0.15,0.62,38.63,XY25,NTT,FINANCE
74,TITAGARH,1548.00,1.18,69.0,H-SC,3.16,165830.0,-26185.0,105070.0,0.12,-13.64,63.36,41.08,158.0,-0.25,1.21,40.52,XY24,NTT,ENGINEERING
7,ATULAUTO,844.00,3797.30,59.0,M-SC,5.38,121564.0,-25804.0,77619.0,-2.47,-17.51,63.85,35.16,236.0,-0.33,0.89,24.17,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.07
1,25,43.99
2,50,74.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.55
LC    32.10
MC    23.36
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.35
X40      15.37
X40N     11.96
XR       11.78
AR        8.85
XY25      8.25
OX40N     8.21
X200      1.81
MH        1.70
X5K       1.44
SR        1.29
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.34
H-LC    25.37
H-MC    20.34
M-SC    13.63
M-LC     5.66
M-MC     2.71
L-SC     1.58
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.71,-12.16,73.31
FMCG,11.93,-3.91,40.26
FINANCE,9.75,-15.33,60.50
MISC,7.43,-13.15,76.05
BANKS,6.37,-14.59,74.87
PAINTS,5.85,-15.51,32.73
ELECTRICAL,5.76,-4.52,62.13
AC,3.72,4.58,11.40
INSURANCE,3.44,-7.07,43.79


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2832932.0,22
XR,1297362.0,14
AR,1289031.0,9
X40,746646.0,10
X40N,570925.0,9
OX40N,525035.0,10
XY25,338089.0,6
SR,246021.0,2
X5K,120793.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3402254.0,29
M-SC,1381431.0,17
H-LC,1224577.0,14
H-MC,1090941.0,15
M-LC,376738.0,4
M-MC,306634.0,2
L-SC,244589.0,3
L-MC,60263.0,1
L-LC,38447.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1024103.0      6
M-SC       XY24       799703.0      7
H-SC       AR         775616.0      5
           XR         752941.0      7
H-MC       XY24       486723.0      4
H-LC       X40        462789.0      5
           AR         351898.0      2
M-MC       XY24       306634.0      2
H-SC       X40N       287971.0      4
           OX40N      252005.0      4
           SR         246021.0      2
H-MC       X40        212883.0      4
H-LC       X40N       209867.0      3
L-SC       XR         161635.0      2
M-SC       AR         161517.0      2
M-LC       XY24       154839.0      2
H-MC       XY25       127639.0      2
M-SC       OX40N      124863.0      4
M-LC       X5K        120793.0      1
M-SC       XR         114168.0      2
           XY25       110206.0      1
H-MC       XR         107249.0      1
M-LC       XR         101106.0      1
L-SC       OX40N       82954.0      1
H-LC       X200        77296.0      1
H-MC       X40N        73087.0      2
M-SC       X40         70974.0      1
H-MC       OX40N       65213.0      1
H-SC       MH          63597.0      1
H-LC       XY25        61797.0      2
           XY24        60930.0      1
L-MC       XR          60263.0      1
L-LC       XY25        38447.0      1
H-MC       MH          18147.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
